In [1]:
from highcharts import Highmap
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import requests

from hc_keys import hc_keys

### Get population by US county from the US Census API

In [2]:
# API key requested from www.census.gov
f = open('/Users/brianna/Dropbox/data_project/census_api_key.txt', 'r')
api_key = f.readline()[:-1]
f.close()

In [3]:
# Get population by US county
url = 'http://api.census.gov/data/2015/pep/population?get=POP,GEONAME&for=%s:*&DATE=8&key=%s' % ('county', api_key)
response = requests.get(url)

if response.status_code != 200:
    raise ValueError('Unexpected status code: %s' % response.status_code)
else:
    data = eval(response.content)
    headers = data.pop(0)
    population = pd.DataFrame(data, columns=headers)

### Remove non-voting states

In [4]:
print('%s unique states' % population.state.nunique())
print('Remove Puerto Rico (72) and District of Columbia (11)')
pop_df = population.drop(population[population.state.isin(['11', '72'])].index)
print('%s unique states' % pop_df.state.nunique())

52 unique states
Remove Puerto Rico (72) and District of Columbia (11)
50 unique states


### Split GEONAME into County, State, Division, Region, Country

In [5]:
pop_df.rename(columns={'state': 'state_no', 'county': 'county_no', 'POP': 'population'}, inplace=True)
pop_df['county'] = pop_df.GEONAME.apply(lambda x: x.split(', ')[0])
pop_df['state'] = pop_df.GEONAME.apply(lambda x: x.split(', ')[1])
pop_df['division'] = pop_df.GEONAME.apply(lambda x: x.split(', ')[2])
pop_df['region'] = pop_df.GEONAME.apply(lambda x: x.split(', ')[3])

In [6]:
# Add hc-keys for plotting in Highcharts
pop_df['hc_key'] = pop_df.state.apply(lambda x: hc_keys[x])

In [7]:
# Convert population (POP) from string to int
pop_df['population'] = pop_df.population.apply(lambda x: int(x))

### Get population by state

In [8]:
state_pop = pop_df.groupby('hc_key').sum()

In [9]:
# state_pop.head()

In [10]:
state_pop_data = []

for k in state_pop.index:
    state_pop_data.append({'hc-key': k, 'value': state_pop.population[k]})

### Plot population by state

In [65]:
options = { # construct option dict
                                   
    'chart' :{ 'renderTo' : 'container'
    },
                           
    'title' : {
        'text' : 'US Population by State'
    },

    'mapNavigation': {
        'enabled': True,
        'buttonOptions': {
            'verticalAlign': 'bottom'
        }
    },
    'colorAxis': {
                'type': 'logarithmic',
                'min': 5e5,
                'minColor': '#FFFFFF', # white
                'maxColor': '#BC0AD1', # dark magenta
#                 'maxColor': '#699C02', # dark green      
            },
} 

In [66]:
H = Highmap(width = 650, height = 500)

H.set_dict_options(options) # set options
H.add_data_set(state_pop_data, 'map', 'Population', joinBy='hc-key', # set dataset [label string in json, label string in local dicts]
                states={
                    'hover': {
                        'color': '#FFC300'
                    }
                },
                dataLabels={
                    'enabled': True,
                    'format': '{point.properties.postal}'
                })


# source is javascript link from http://code.highcharts.com/mapdata/
# H.set_map_source('http://code.highcharts.com/mapdata/countries/us/custom/us-all-mainland.js')
H.set_map_source('http://code.highcharts.com/mapdata/countries/us/us-all.js')

H